In [1]:
from googlesearch import search
from bs4 import BeautifulSoup
import concurrent.futures
from urllib.parse import urlparse, urljoin
import urllib.robotparser
import urllib.request
import queue

In [2]:
def google_search(query):
    results_list = []
    for result in search(query,        # The query you want to run
                    tld = 'com',  # The top level domain
                    lang = 'en',  # The language
                    num = 10,     # Number of results per page
                    start = 0,    # First result to retrieve
                    stop = 10,  # Last result to retrieve
                
                ):
        results_list.append(result)
    return results_list

In [3]:
def create_robot_parser(url):
    rp = urllib.robotparser.RobotFileParser()
    rp.set_url(url+"/robots.txt")
    rp.read()
    return rp

rp=create_robot_parser("https://engineering.nyu.edu/")
print(rp.can_fetch("*", "https://engineering.nyu.edu/academics/programs"))
print(rp.can_fetch("*", "https://engineering.nyu.edu/profiles/"))

True
False


In [6]:
def crawl(url, index):
    try:
        page=urllib.request.urlopen(url)
    except urllib.error.HTTPError as error:
        print(index, url+" "+str(error.code))
        return None
    except urllib.error.URLError:
        print(index, url+" "+"URLError")
        return None
    content=page.read()
    size=len(content)/1024
    print(index, url+" Size: "+"{:.2f}".format(size)+"kb")
    return content,url
# print(crawl("http://mit.edu",1))
# print(crawl("https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html",2))

In [7]:
def get_links(page_content,url):
    if page_content is None:
        return []
    soup = BeautifulSoup(page_content, "html.parser")

    base = soup.find_all('base')
    linkset=set()
    if base:
        url=base[0].get('href')
    base_parse_result=urlparse(url)
    current_site=base_parse_result.scheme+"://"+base_parse_result.netloc
    for link in soup.find_all('a',href=True):
        #filter the links
        #non-url
        #modify relative url to common url
        href=link.get('href')
        parse_result=urlparse(href)
        robot_parser=create_robot_parser(current_site)
        if not href:
            continue
        if href[0]!="/" and href[0]!="#":
            if href[:4] != "http":
                continue
        if parse_result.scheme=="":
            link_to_add=urljoin(url,href)
        else:
            link_to_add=href
        # print(link_to_add)
        if robot_parser.can_fetch("*",link_to_add):
            linkset.add(link_to_add)
            # print(href)
    return list(linkset)
# content,url=crawl("http://engineering.nyu.edu",1)
# print(content)
# print(len(get_links(content,url)))
# get_links('http://nyu.edu')
# get_links("https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html")

In [8]:
robot_parser=create_robot_parser("https://www.nationalgridus.com")
robot_parser.can_fetch("*","https://www1.nationalgridus.com/UsageCostGraphElectric")

False

In [9]:
seen=set()
scheduled_crawl=[]
crawled_site={}
q=queue.Queue()
robot_parsers={}
counter=0
results_list=google_search("brooklyn union")
batch=queue.Queue()
result_count=0
qsize=0
for link in results_list:
    parse_result=urlparse(link)
    parsed_link=urljoin(parse_result.scheme+"://"+parse_result.netloc,parse_result.path)
    netloc=parse_result.netloc
    if parsed_link not in seen:
        if netloc not in crawled_site or crawled_site[netloc]<10:
            q.put(link)
            qsize+=1
            seen.add(link)
            if netloc not in crawled_site:
                crawled_site[netloc]=1
            else:
                crawled_site[netloc]+=1


In [10]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in range(10):
        cur_url=q.get()
        qsize-=1
        # print("putting into batch",counter,cur_url)
        # print(crawl(cur))
        scheduled_crawl.append((counter,cur_url))
        counter+=1
    batch_results=[executor.submit(crawl,url,index) for index,url in scheduled_crawl]
    for job in concurrent.futures.as_completed(batch_results):
        crawl_result=job.result()
        if crawl_result is not None:
            batch.put(crawl_result)
        result_count+=1



2 https://www.nationalgridus.com/contact-us Size: 119.30kb
3 https://www.nationalgridus.com/NY-Home/ Size: 125.61kb
0 https://www.nationalgridus.com/ Size: 118.49kb
4 https://www.nationalgridus.com/regionalized/Billing-Payments/Ways-to-Pay Size: 118.49kb
9 https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html 403
6 https://twitter.com/BrooklynPatch/status/1438255356694736901?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet Size: 78.35kb
7 https://twitter.com/NYDailyNews/status/1438255313698820097?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet Size: 77.94kb
5 https://twitter.com/NYCTSubway/status/1438256079339769860?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet Size: 77.89kb
8 https://twitter.com/search?q=brooklyn+union&ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Esearch Size: 77.29kb
1 https://www1.nationalgridus.com/signin Size: 167.03kb


In [11]:
result_count

10

In [12]:

while not batch.empty():
    crawled_content=batch.get()
    
    if not crawled_content:
        continue
    print(len(crawled_content))
    print("go")
    content,url=crawled_content[0],crawled_content[1]
    print(len(content),url)
    for link in get_links(content,url):
        if link:

            parse_result=urlparse(link)
            netloc=parse_result.netloc
            if link not in seen:
                #up to 10 page from a netloc can be crawled
                if netloc not in crawled_site or crawled_site[netloc]<10:
                    q.put(link)
                    print(link)
                    qsize+=1
                    seen.add(link)
                    if netloc not in crawled_site:
                        crawled_site[netloc]=1
                    else:
                        crawled_site[netloc]+=1
    print("qsize:", qsize)
    if qsize>=result_count:
        break
while not q.empty():
    print(q.get())

2
go
122168 https://www.nationalgridus.com/contact-us
qsize: 0
2
go
128627 https://www.nationalgridus.com/NY-Home/
qsize: 0
2
go
121331 https://www.nationalgridus.com/
qsize: 0
2
go
121331 https://www.nationalgridus.com/regionalized/Billing-Payments/Ways-to-Pay
qsize: 0
2
go
80229 https://twitter.com/BrooklynPatch/status/1438255356694736901?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet
https://business.twitter.com/en/help/troubleshooting/how-twitter-ads-work.html?ref=web-twc-ao-gbl-adsinfo&utm_source=twc&utm_medium=web&utm_campaign=ao&utm_content=adsinfo
https://support.twitter.com/articles/20170514
https://twitter.com/tos
https://twitter.com/privacy
https://legal.twitter.com/imprint
https://help.twitter.com/using-twitter/twitter-supported-browsers
qsize: 6
2
go
79810 https://twitter.com/NYDailyNews/status/1438255313698820097?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet
qsize: 6
2
go
79759 https://twitter.com/NYCTSubway/status/1438256079339769860?ref_src=twsrc%5Egoogle

In [55]:
def bfs_crawler(query):
    seen=set()
    scheduled_crawl=[]
    crawled_site={}
    q=queue.Queue()
    robot_parsers={}
    counter=0
    results_list=google_search(query)
    batch=queue.Queue()
    result_count=0
    qsize=0
    for link in results_list:
        parse_result=urlparse(link)
        parsed_link=urljoin(parse_result.scheme+"://"+parse_result.netloc,parse_result.path)
        netloc=parse_result.netloc
        if parsed_link not in seen:
            if netloc not in crawled_site or crawled_site[netloc]<10:
                q.put(link)
                qsize+=1
                seen.add(link)
                if netloc not in crawled_site:
                    crawled_site[netloc]=1
                else:
                    crawled_site[netloc]+=1
    with concurrent.futures.ThreadPoolExecutor() as executor:
        while result_count<500 and not q.empty():
            #every batch double in size
            for i in range(max(4,result_count)):
                cur_url=q.get()
                qsize-=1
                # print("putting into batch",counter,cur_url)
                # print(crawl(cur))
                scheduled_crawl.append((counter,cur_url))
                counter+=1
            batch_results=[executor.submit(crawl,url,index) for index,url in scheduled_crawl]
            for job in concurrent.futures.as_completed(batch_results):
                crawl_result=job.result()
                batch.put(crawl_result)
                result_count+=1
            while not batch.empty() and q.qsize()<result_count:
                crawled_content=batch.get()
                if not crawled_content:
                    continue
                for link in get_links(crawled_content[0],crawled_content[1]):
                    if link:
                        parse_result=urlparse(link)
                        # parsed_link=urljoin(parse_result.scheme+"://"+parse_result.netloc,parse_result.path)
                        netloc=parse_result.netloc
                        if link not in seen:
                            #up to 10 page from a netloc can be crawled
                            if netloc not in crawled_site or crawled_site[netloc]<10:
                                # current_site=parse_result.scheme+"://"+parse_result.netloc
                                # if current_site not in robot_parsers:
                                #     robot_parsers[current_site]=create_robot_parser(current_site)
                                # robot_parser=robot_parsers[current_site]
                                # if robot_parser.can_fetch("*",link):
                                q.put(link)
                                qsize+=1
                                print(link)
                                print("qsize:",qsize)
                                seen.add(link)
                                if netloc not in crawled_site:
                                    crawled_site[netloc]=1
                                else:
                                    crawled_site[netloc]+=1
                if qsize>=result_count:
                    break
    
        
bfs_crawler("brooklyn union")

2 https://www.nationalgridus.com/contact-us Size: 119.30kb
3 https://www.nationalgridus.com/NY-Home/ Size: 125.61kb
0 https://www.nationalgridus.com/ Size: 118.49kb
1 https://www1.nationalgridus.com/signin Size: 167.03kb
2 https://www.nationalgridus.com/contact-us Size: 119.30kb
3 https://www.nationalgridus.com/NY-Home/ Size: 125.61kb
0 https://www.nationalgridus.com/ Size: 118.49kb
4 https://www.nationalgridus.com/regionalized/Billing-Payments/Ways-to-Pay Size: 118.49kb
7 https://twitter.com/NYDailyNews/status/1438255313698820097?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet Size: 77.94kb
6 https://twitter.com/BrooklynPatch/status/1438255356694736901?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet Size: 78.19kb
5 https://twitter.com/NYCTSubway/status/1438256079339769860?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet Size: 77.89kb
1 https://www1.nationalgridus.com/signin Size: 167.03kb
https://www1.nationalgridus.com:443/Outages/SignIn
qsize: 3
https://www1.nationa

ValueError: unknown url type: ':///robots.txt'

In [ ]:
#priority queue
#class ScoredUrl

novelty_res={}
candidate_urls={}
url_pos={}
class ScoredUrl:
    def __init__(self, url):
        self.url=url
        self.netloc=urlparse(url).netloc
        self.importance=0
        if self.netloc not in novelty_res:
            novelty_res[self.netloc]=10

    def __eq__(self,other):
        return novelty_res[self.netloc]+self.importance==novelty_res[other.netloc]+other.importance

    def __lt__(self,other):
        return novelty_res[self.netloc]+self.importance<novelty_res[other.netloc]+other.importance

    def __gt__(self,other):
        return self>other

    def add_importance(self):
        self.importance+=1

    def crawl(self):
        return get_links(self.url)



class UrlQueue:
    def __init__(self):
        self.res=[]
    
    def add(self,new_url):
        heapq.heappush(self.res,new_url)
    
    def update(self,new_url):
        while
    #update

SyntaxError: invalid syntax (Temp/ipykernel_21872/177025105.py, line 40)